<a href="https://colab.research.google.com/github/Spandana825/cybersecproject/blob/main/Copy_of_level1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install whois

In [ ]:
pip install pyquery

In [ ]:
import whois
from datetime import datetime, timezone
import math
import pandas as pd
import numpy as np
from pyquery import PyQuery
from requests import get

class UrlFeaturizer(object):
    def __init__(self, url):
        self.url = url
        self.domain = url.split('//')[-1].split('/')[0]
        self.today = datetime.now().replace(tzinfo=None)

        try:
            self.whois = whois.query(self.domain).__dict__
        except:
            self.whois = None

        try:
            self.response = get(self.url)
            self.pq = PyQuery(self.response.text)
        except:
            self.response = None
            self.pq = None

    ## URL string Features
    def entropy(self):
        string = self.url.strip()
        prob = [float(string.count(c)) / len(string) for c in dict.fromkeys(list(string))]
        entropy = sum([(p * math.log(p) / math.log(2.0)) for p in prob])
        return entropy

    def ip(self):
        string = self.url
        flag = False
        if ("." in string):
            elements_array = string.strip().split(".")
            if(len(elements_array) == 4):
                for i in elements_array:
                    if (i.isnumeric() and int(i)>=0 and int(i)<=255):
                        flag=True
                    else:
                        flag=False
                        break
        if flag:
            return 1
        else:
            return 0

    def numDigits(self):
        digits = [i for i in self.url if i.isdigit()]
        return len(digits)

    def urlLength(self):
        return len(self.url)

    def numParameters(self):
        params = self.url.split('&')
        return len(params) - 1

    def numFragments(self):
        fragments = self.url.split('#')
        return len(fragments) - 1

    def numSubDomains(self):
        subdomains = self.url.split('http')[-1].split('//')[-1].split('/')
        return len(subdomains)-1

    def domainExtension(self):
        ext = self.url.split('.')[-1].split('/')[0]
        return ext

    ## URL domain features
    def hasHttp(self):
        return 'http:' in self.url

    def hasHttps(self):
        return 'https:' in self.url

    def daysSinceRegistration(self):
        if self.whois and self.whois['creation_date']:
            diff = self.today - self.whois['creation_date'].replace(tzinfo=None)
            diff = str(diff).split(' days')[0]
            return diff
        else:
            return 0

    def daysSinceExpiration(self):
        if self.whois and self.whois['expiration_date']:
            diff = self.whois['expiration_date'].replace(tzinfo=None) - self.today
            diff = str(diff).split(' days')[0]
            return diff
        else:
            return 0

     ## URL Page Features
    def bodyLength(self):
        if self.pq is not None:
            return len(self.pq('html').text()) if self.urlIsLive else 0
        else:
            return 0

    def numTitles(self):
        if self.pq is not None:
            titles = ['h{}'.format(i) for i in range(7)]
            titles = [self.pq(i).items() for i in titles]
            return len([item for s in titles for item in s])
        else:
            return 0

    def numImages(self):
        if self.pq is not None:
            return len([i for i in self.pq('img').items()])
        else:
            return 0

    def numLinks(self):
        if self.pq is not None:
            return len([i for i in self.pq('a').items()])
        else:
            return 0

    def scriptLength(self):
        if self.pq is not None:
            return len(self.pq('script').text())
        else:
            return 0

    def specialCharacters(self):
        if self.pq is not None:
            bodyText = self.pq('html').text()
            schars = [i for i in bodyText if not i.isdigit() and not i.isalpha()]
            return len(schars)
        else:
            return 0

    def scriptToSpecialCharsRatio(self):
        v = self.specialCharacters()
        if self.pq is not None and v!=0:
            sscr = self.scriptLength()/v
        else:
            sscr = 0
        return sscr

    def scriptTobodyRatio(self):
        v = self.bodyLength()
        if self.pq is not None and v!=0:
            sbr = self.scriptLength()/v
        else:
            sbr = 0
        return sbr

    def bodyToSpecialCharRatio(self):
        v = self.bodyLength()
        if self.pq is not None and v!=0:
            bscr = self.specialCharacters()/v
        else:
            bscr = 0
        return bscr

    def urlIsLive(self):
        return self.response == 200

    def run(self):
        data = {}
        data['entropy'] = self.entropy()
        data['numDigits'] = self.numDigits()
        data['urlLength'] = self.urlLength()
        data['numParams'] = self.numParameters()
        data['hasHttp'] = self.hasHttp()
        data['hasHttps'] = self.hasHttps()
        data['urlIsLive'] = self.urlIsLive()
        data['bodyLength'] = self.bodyLength()
        data['numTitles'] = self.numTitles()
        data['numImages'] = self.numImages()
        data['numLinks'] = self.numLinks()
        data['scriptLength'] = self.scriptLength()
        data['specialChars'] = self.specialCharacters()
        data['ext'] = self.domainExtension()
        data['dsr'] = self.daysSinceRegistration()
        data['dse'] = self.daysSinceExpiration()
        data['sscr'] = self.scriptToSpecialCharsRatio()
        data['sbr'] = self.scriptTobodyRatio()
        data['bscr'] = self.bodyToSpecialCharRatio()
        data['num_%20'] = self.url.count("%20")
        data['num_@'] = self.url.count("@")
        data['has_ip'] = self.ip()

        return data

In [ ]:
# Assuming the UrlFeaturizer class is defined as provided

# Example usage:
url_to_featurize = "https://www.google.com"
featurizer = UrlFeaturizer(url_to_featurize)

# Run the featurizer to get the features
features = featurizer.run()

# Display the features
for key, value in features.items():
    print(f"{key}: {value}")


entropy: -3.663532754804254
numDigits: 0
urlLength: 22
numParams: 0
hasHttp: False
hasHttps: True
urlIsLive: False
bodyLength: 14746
numTitles: 0
numImages: 1
numLinks: 17
scriptLength: 13021
specialChars: 4518
ext: com
dsr: 0
dse: 0
sscr: 2.8820274457724655
sbr: 0.8830191238301912
bscr: 0.3063881730638817
num_%20: 0
num_@: 0
has_ip: 0


In [ ]:
order = ['bodyLength', 'bscr', 'dse', 'dsr', 'entropy', 'hasHttp', 'hasHttps',
       'has_ip', 'numDigits', 'numImages', 'numLinks', 'numParams',
       'numTitles', 'num_%20', 'num_@', 'sbr', 'scriptLength', 'specialChars',
       'sscr', 'urlIsLive', 'urlLength']


test = []
for i in order:
		test.append(features[i])
print(test)

[14746, 0.3063881730638817, 0, 0, -3.663532754804254, False, True, 0, 0, 1, 17, 0, 0, 0, 0, 0.8830191238301912, 13021, 4518, 2.8820274457724655, False, 22]


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

data = pd.read_csv("/content/sample_data/feature.csv")
data.replace(True,1,inplace = True)
data.replace(False,0,inplace = True)

y = data["File"]
data = data.drop(columns = "File")

encoder = LabelEncoder()
encoder.fit(y)
Y = encoder.transform(y)
# Create a mapping between original labels and their encoded values
label_mapping = dict(zip(y, Y))

# Print original labels and their encoded forms
for label, encoded_value in label_mapping.items():
    print(f"Label: {label}, Encoded Value: {encoded_value}")

# Print column names before scaling
print("\nColumns before scaling:")
print(data.columns)
data = data.drop(data.columns[0], axis=1)

scaler = MinMaxScaler(feature_range=(0, 1))
X = pd.DataFrame(scaler.fit_transform(data))
print("\nColumns after scaling:")
print(X.columns)


num_columns = len(X.columns)
print(f"Number of columns in the DataFrame: {num_columns}")

Label: spam_dataset, Encoded Value: 4
Label: Benign_list_big_final, Encoded Value: 0
Label: phishing_dataset, Encoded Value: 3
Label: Malware_dataset, Encoded Value: 2
Label: Defacement, Encoded Value: 1

Columns before scaling:
Index(['Unnamed: 0', 'bodyLength', 'bscr', 'dse', 'dsr', 'entropy', 'hasHttp',
       'hasHttps', 'has_ip', 'numDigits', 'numImages', 'numLinks', 'numParams',
       'numTitles', 'num_%20', 'num_@', 'sbr', 'scriptLength', 'specialChars',
       'sscr', 'urlIsLive', 'urlLength'],
      dtype='object')

Columns after scaling:
RangeIndex(start=0, stop=21, step=1)
Number of columns in the DataFrame: 21


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


**DECISION TREE**

In [ ]:
#importing packages
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
# Decision Tree model
from sklearn.tree import DecisionTreeClassifier

# instantiate the model with check_input set to False
tree = DecisionTreeClassifier(max_depth=5)

# fit the model
tree.fit(X_train, y_train)

#predicting the target value from the model for the samples
y_test_tree = tree.predict(X_test)
y_train_tree = tree.predict(X_train)

#computing the accuracy of the model performance
acc_train_tree = accuracy_score(y_train,y_train_tree)
acc_test_tree = accuracy_score(y_test,y_test_tree)
print("Decision Tree: Accuracy on training Data: {:.3f}".format(acc_train_tree))
print("Decision Tree: Accuracy on test Data: {:.3f}".format(acc_test_tree))
#computing the accuracy of the model performance
precision_train_tree = precision_score(y_train, y_train_tree, average='weighted')
precision_test_tree = precision_score(y_test, y_test_tree, average='weighted')

print("Decision Tree: Precision on training Data: {:.3f}".format(precision_train_tree))
print("Decision Tree: Precision on test Data: {:.3f}".format(precision_test_tree))


Decision Tree: Accuracy on training Data: 0.662
Decision Tree: Accuracy on test Data: 0.670
Decision Tree: Precision on training Data: 0.721
Decision Tree: Precision on test Data: 0.725


**RANDOM FOREST**

In [ ]:
# Random Forest model
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score

# instantiate the model
forest = RandomForestClassifier(max_depth=5)

# fit the model
forest.fit(X_train, y_train)

#predicting the target value from the model for the samples
y_test_forest = forest.predict(X_test)
y_train_forest = forest.predict(X_train)

#computing the accuracy of the model performance
acc_train_forest = accuracy_score(y_train,y_train_forest)
acc_test_forest = accuracy_score(y_test,y_test_forest)

print("Random forest: Accuracy on training Data: {:.3f}".format(acc_train_forest))
print("Random forest: Accuracy on test Data: {:.3f}".format(acc_test_forest))

#computing the precison of the model performance
precision_train_forest = precision_score(y_train, y_train_forest, average='weighted')
precision_test_forest = precision_score(y_test, y_test_forest, average='weighted')
print("Random Forest: Precision on training Data: {:.3f}".format(precision_train_forest))
print("Random Forest: Precision on test Data: {:.3f}".format(precision_test_forest))

Random forest: Accuracy on training Data: 0.726
Random forest: Accuracy on test Data: 0.734
Random Forest: Precision on training Data: 0.765
Random Forest: Precision on test Data: 0.772


**LOGISTIC REGRESSION**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')

# Initialize the Logistic Regression model
logistic_model = LogisticRegression()

# Fit the model on the training data
logistic_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = logistic_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
precision = precision_score(y_test, y_pred, average='weighted')
print(f"Precision: {precision}")

Accuracy: 0.5686256532502063
Precision: 0.5865382666419349


**KNN**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

# Create a KNN classifier with a specified number of neighbors (e.g., 3)
knn_classifier = KNeighborsClassifier(n_neighbors=3)

# Train the classifier on the training data
knn_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = knn_classifier.predict(X_test)

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
precision = precision_score(y_test, y_pred, average='weighted')
print(f"Precision: {precision}")


Accuracy: 0.8787017511689741
Precision: 0.8781134996784091


**GRADIENT BOOSTING CLASSIFIER**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score

# Create a GradientBoostingClassifier instance
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Fit the model on the training data
clf.fit(X_train, y_train)

# Make predictions on the test data
predictions = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

# Calculate precision
precision = precision_score(y_test, predictions, average='weighted')
print(f"Precision: {precision}")

Accuracy: 0.8398276336297791
Precision: 0.8397522131598922


**ENSEMBLED**

FINAL OUTPUT

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

# Assume you have your dataset and labels
# X_train, X_test, y_train, y_test = ...

# Create individual models
model1 = DecisionTreeClassifier()
model2 = SVC(probability=True)  # Note: probability=True for using with AdaBoost
model3 = RandomForestClassifier()

# Specify base learner for AdaBoost
base_model = DecisionTreeClassifier()

# Create an ensemble using AdaBoost
ensemble_model = AdaBoostClassifier(
    base_estimator=base_model,
    n_estimators=50,  # Number of boosting rounds
    learning_rate=1.0  # Weight applied to each model's contribution
)

# Train the ensemble model
ensemble_model.fit(X_train, y_train)

# Make predictions
ensemble_predictions = ensemble_model.predict(X_test)

# Evaluate the ensemble model
accuracy = accuracy_score(y_test, ensemble_predictions)
print(f'Ensemble Model Accuracy: {accuracy}')
precision = precision_score(y_test, ensemble_predictions, average='weighted')
print(f"Ensemble Model Precision: {precision}")


url_to_featurize = "espn.go.com/nba/player/_/id/3457/brandon-rush"
featurizer = UrlFeaturizer(url_to_featurize)
features = featurizer.run()
del features['ext']

order = ['bodyLength', 'bscr', 'dse', 'dsr', 'entropy', 'hasHttp', 'hasHttps',
         'has_ip', 'numDigits', 'numImages', 'numLinks', 'numParams',
         'numTitles', 'num_%20', 'num_@', 'sbr', 'scriptLength', 'specialChars',
         'sscr', 'urlIsLive', 'urlLength']

# test = []
# for i in order:
# 		test.append(features[i])
# print(test)

modified_dict = {key: features.get(key, None) for key in order}

# print("Number of features in order:", len(order))
# print("Number of features in features dictionary:", len(modified_dict))


feature_array = np.array([modified_dict.get(key, None) for key in order]).reshape(1, -1)

# Replace 'tree' with your actual trained decision tree model
predicted_label = ensemble_model.predict(feature_array)

# Display the predicted label
print("Predicted Label:", predicted_label[0])

Ensemble Model Accuracy: 0.8925460713303384
Ensemble Model Precision: 0.8918383045345821
Predicted Label: 3


OTHER TRIED ENSEMBLED MODELS

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

# Create individual models
model1 = LogisticRegression()
model2 = KNeighborsClassifier()
model3 = GradientBoostingClassifier()

# Create an ensemble of models
ensemble_model = VotingClassifier(estimators=[
    ('lr', model1),
    ('knn', model2),
    ('gb', model3)
], voting='soft')  # 'soft' for soft voting, 'hard' for hard voting

# Train the ensemble model
ensemble_model.fit(X_train, y_train)

# Make predictions
ensemble_predictions = ensemble_model.predict(X_test)

# Evaluate the ensemble model
accuracy = accuracy_score(y_test, ensemble_predictions)
print(f'Ensemble Model Accuracy: {accuracy}')
precision = precision_score(y_test, ensemble_predictions, average='weighted')
print(f"Ensemble Model Precision: {precision}")


Ensemble Model Accuracy: 0.8831942789034565
Ensemble Model Precision: 0.8831788019932503


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

# Assuming you have your data in X_train, X_test, y_train, y_test

# Split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Define three Gradient Boosting models
model1 = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model2 = GradientBoostingClassifier(n_estimators=150, learning_rate=0.05, max_depth=2, random_state=42)
model3 = GradientBoostingClassifier(n_estimators=120, learning_rate=0.08, max_depth=4, random_state=42)

# Create an ensemble using VotingClassifier
ensemble_model = VotingClassifier(estimators=[('model1', model1), ('model2', model2), ('model3', model3)], voting='hard')

# Fit the ensemble model on the training data
ensemble_model.fit(X_train, y_train)

# Make predictions on the test set
ensemble_predictions = ensemble_model.predict(X_test)

# Evaluate the accuracy of the ensemble model
accuracy = accuracy_score(y_test, ensemble_predictions)
print(f'Ensemble Accuracy: {accuracy}')
precision = precision_score(y_test, ensemble_predictions, average='weighted')
print(f"Ensemble Model Precision: {precision}")



Ensemble Accuracy: 0.8412945814614468
Ensemble Model Precision: 0.8413692457415114


**PEDICTION FOR URL**

In [ ]:
url_to_featurize = "espn.go.com/nba/player/_/id/3457/brandon-rush"
featurizer = UrlFeaturizer(url_to_featurize)
features = featurizer.run()
del features['ext']

order = ['bodyLength', 'bscr', 'dse', 'dsr', 'entropy', 'hasHttp', 'hasHttps',
         'has_ip', 'numDigits', 'numImages', 'numLinks', 'numParams',
         'numTitles', 'num_%20', 'num_@', 'sbr', 'scriptLength', 'specialChars',
         'sscr', 'urlIsLive', 'urlLength']

# test = []
# for i in order:
# 		test.append(features[i])
# print(test)

modified_dict = {key: features.get(key, None) for key in order}

# print("Number of features in order:", len(order))
# print("Number of features in features dictionary:", len(modified_dict))


feature_array = np.array([modified_dict.get(key, None) for key in order]).reshape(1, -1)

# Replace 'tree' with your actual trained decision tree model
predicted_label = logistic_model.predict(feature_array)

# Display the predicted label
print("Predicted Label:", predicted_label[0])

Predicted Label: 0


In [ ]:
!pip install ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.6 MB/s eta 0:00:00


In [ ]:


import ipywidgets as widgets
from IPython.display import display

def process_url(change):
    url_to_featurize = url_input.value
    featurizer = UrlFeaturizer(url_to_featurize)
    features = featurizer.run()

    # Call your model for prediction
    order = ['bodyLength', 'bscr', 'dse', 'dsr', 'entropy', 'hasHttp', 'hasHttps',
         'has_ip', 'numDigits', 'numImages', 'numLinks', 'numParams',
         'numTitles', 'num_%20', 'num_@', 'sbr', 'scriptLength', 'specialChars',
         'sscr', 'urlIsLive', 'urlLength']

    modified_dict = {key: features.get(key, None) for key in order}

    feature_array = np.array([modified_dict.get(key, None) for key in order]).reshape(1, -1)

    # Replace 'tree' with your actual trained decision tree model
    predicted_label = ensemble_model.predict(feature_array)
    is_malicious = predicted_label[0]  # Replace with your actual model prediction function

    # Display features and result
    output_text.value = "Features:\n" + "\n".join([f"{key}: {value}" for key, value in features.items()]) + f'\n\nIs Malicious: {is_malicious}'

# URL Input
url_label = widgets.Label(value="Enter URL:")
url_input = widgets.Text(placeholder="Enter URL", description="")
url_input.observe(process_url, names='value')

# Button to process URL
process_button = widgets.Button(description="Process URL")
process_button.on_click(process_url)

# Output Display
output_text = widgets.HTML(value="Output:")

# Arrange widgets in a layout
input_box = widgets.VBox([url_label, url_input, process_button])
output_box = widgets.VBox([output_text])
ui = widgets.HBox([input_box, output_box])

# Display the UI
display(ui)
